# Hybrid Univariate #2

We are training a seperate model for each timestep in the lookahead window; univariate input and output
We are feeding the output of the previous model to each succesive model

We are using z-score noralization as opposed to min/max, callbacks, and a bigger dataset, versus the first iteration of this model

After reviewing validation metrics, metrics and training loss did not decline after the first lookahead. This was due to an error in code that generates the training examples for succesive lookaheads. This is addressed in the third iteration

## Load functions and data

In [1]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import mean
from numpy import std
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential
from keras import callbacks
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt



In [2]:
def split_sequence(sequence, n_steps_in, n_steps_out, step_interval, n_step_lookahead):
    X, y = list(), list()
    example_count = int((len(sequence)/step_interval))
    for i in range(example_count):
        # find the end of this pattern
        end_ix = (i*step_interval) + n_steps_in
        out_start_ix = end_ix + n_step_lookahead -1
        out_end_ix = end_ix + n_steps_out + n_step_lookahead -1
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[(i*step_interval):end_ix], sequence[out_start_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [3]:
#To demonstrate above function
sequence = range(0,13)
n_steps_in = 1
n_steps_in = 5
n_steps_out =1
step_interval =1
n_step_lookahead=5
split_sequence(sequence, n_steps_in, n_steps_out, step_interval, n_step_lookahead)

(array([[0, 1, 2, 3, 4],
        [1, 2, 3, 4, 5],
        [2, 3, 4, 5, 6],
        [3, 4, 5, 6, 7]]),
 array([[ 9],
        [10],
        [11],
        [12]]))

In [4]:
percentile_data = pd.read_csv (r'C:\Users/conal/Desktop/MCM/Practicum - Copy/data/block gas price percentile data.csv', header=0)
percentile_data['datetime'] = pd.to_datetime(percentile_data['block_timestamp'], format = '%Y-%m-%d %H:%M:%S UTC')

percentile_data = percentile_data.sort_values(by='datetime',ascending=False)
percentile_data = percentile_data.set_index('datetime')
percentile_data = percentile_data.resample('5T').mean()
percentile_data = percentile_data/1000000000

In [5]:
usage_data = pd.read_csv (r'C:\Users\conal\Desktop\MCM\Practicum - Copy\data\ETH,gas,usage merged 11-26 to 05-26.csv', header=0)
usage_data['datetime'] = pd.to_datetime(usage_data['datetime'], format = '%Y-%m-%d %H:%M:%S')
usage_data = usage_data.set_index('datetime')

usage_data = usage_data.squeeze()
usage_data = usage_data.astype('float')
usage_data = usage_data.resample('5T').mean()

In [6]:
usage_data2 = pd.read_csv (r'C:\Users\conal\Desktop\MCM\Practicum - Copy\data\Contract counts 2021-11-26 to 2022-05-26.csv', header=0, index_col=0)
usage_data2['datetime'] = pd.to_datetime(usage_data2['block_timestamp'], format = '%Y-%m-%d %H:%M:%S') 
usage_data2 = usage_data2.set_index('datetime')
usage_data2 = usage_data2.drop(['block_timestamp'], axis=1)
usage_data2 = usage_data2.squeeze()
usage_data2 = usage_data2.astype('float')
usage_data2 = usage_data2.resample('5T').sum()

In [7]:
data = usage_data.merge(percentile_data, left_index=True, right_index=True)
data = data.merge(usage_data2, left_index=True, right_index=True)

Load data, datetime to index, downsample with left edge label, convert wei to gwei

In [8]:
def generate_training_val_examples_univariate_output(data):
 
    #Filter inputs, standardize
    data =data[inputs]
    scaler = StandardScaler()
    data[inputs] = scaler.fit_transform(data[inputs])
    

    #Creat input:output examples
    data = data[start_date:end_date].to_numpy()
    X, y = split_sequence(data, n_steps_in, n_steps_out, step_interval, n_step_lookahead)
    
    X_train, X_val = np.split(X, [int(0.7 * len(X))])
    
    y_train, y_val = np.split(y, [int(0.7 * len(X))])

    
    #Reshape to 3D for LSTM, filter output to only the first input
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], len(inputs)))
    y_train =y_train.reshape((y_train.shape[0], y_train.shape[1], len(inputs)))[:,:,:1]
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], len(inputs)))
    y_val = y_val.reshape((y_val.shape[0], y_val.shape[1], len(inputs)))[:,:,:1]

    
    return X_train, y_train, X_val, y_val, scaler


In [9]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error


In [10]:
def descale_y_retrun_metrics(yhat, y_val2):
    #reverts standard scaling, returns dictionary of metrics for each output, for all lookaheads
    dict_indexes=[]
    dict_dfs=[]
    for j in range(0, n_steps_out):
        RMSE_list, MAE_list, MAPE_list, R2_list, MSE_list = [],[],[],[],[]
        for i in range(0, len(inputs)):  
            pred_descaled= (scaler.inverse_transform(yhat[:,j:j+1,:].reshape(yhat.shape[0], yhat.shape[2])))[:, i:i+1]
            groud_truth_descaled= ((scaler.inverse_transform(y_val2[:,j:j+1,:].reshape(y_val2.shape[0], y_val2.shape[2]))))[:, i:i+1]
            RMSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=False)
            MSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=True)
            MAE = mean_absolute_error(groud_truth_descaled, pred_descaled)
            MAPE = mean_absolute_percentage_error(groud_truth_descaled, pred_descaled)
            R2 = r2_score(groud_truth_descaled, pred_descaled)
            RMSE_list.append(RMSE)
            MAE_list.append(MAE)
            MAPE_list.append(MAPE)
            R2_list.append(R2)
            MSE_list.append(MSE)
        metrics_df = pd.DataFrame({'RMSE':RMSE_list, 'MSE':MSE_list, 'MAE':MAE_list, 'MAPE':MAPE_list, 'R2':R2_list}, index=inputs)
        dict_dfs.append(metrics_df)
        dict_indexes.append('Lookahead' +str(j))
    metrics_dict = dict(zip(dict_indexes, dict_dfs))
    return metrics_dict

In [11]:
def descale_y_retrun_metrics_univariate_y(yhat, y_val2):
    #reverts standard scaling, returns dictionary of metrics for single output, for all lookaheads
    dict_indexes=[]
    dict_dfs=[]
    RMSE_list, MAE_list, MAPE_list, R2_list, MSE_list = [],[],[],[],[]
    for j in range(0, n_steps_out):
        pred_descaled= scaler.inverse_transform(yhat)
        groud_truth_descaled= scaler.inverse_transform(y_val2[:,0,])
        RMSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=False)
        MSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=True)
        MAE = mean_absolute_error(groud_truth_descaled, pred_descaled)
        MAPE = mean_absolute_percentage_error(groud_truth_descaled, pred_descaled)
        R2 = r2_score(groud_truth_descaled, pred_descaled)
        RMSE_list.append(RMSE)
        MAE_list.append(MAE)
        MAPE_list.append(MAPE)
        R2_list.append(R2)
        MSE_list.append(MSE)
    metrics_df = pd.DataFrame({'RMSE':RMSE_list, 'MSE':MSE_list, 'MAE':MAE_list, 'MAPE':MAPE_list, 'R2':R2_list}, index=range(1, (n_steps_out+1)))

 
    return metrics_df

In [12]:
def LSTM_model():
    checkpoint_filepath='./Hybrid Univarite2/'
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
    
    model = Sequential()
    model.add(LSTM(units =15,activation='tanh',input_shape=(n_steps_in, len(inputs)),dropout=0.01))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mse')
    return model, model_checkpoint_callback, checkpoint_filepath

## Training loop

In [102]:
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#Create Training Examples for all lookaheads
resample_rate = '5T'
start_date='2021-11-26 00:00:00'
end_date='2021-12-26 23:55:00' 
inputs = ['min_gas_price']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 2016
n_steps_out = 1
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1

y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
training_metrics_dicts=[]
valdiation_metrics_dicts=[]
for month in [0, 1, 2, 3, 4]:
    n_step_lookahead=1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    #generate new training/val data each month
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples_univariate_output(data)
    for i in range(1,11):
        n_step_lookahead = i
        #train model with checkpoint best for best validation loss
        model, model_checkpoint_callback, checkpoint_filepath = LSTM_model()
        train_history = model.fit(X_train, y_train,validation_data=(X_val, y_val), epochs=15, verbose=1, callbacks=[model_checkpoint_callback])
        model.load_weights(checkpoint_filepath)
        model.save(('Hybrid Univarite2/month' +str(month)+'_'+ str(i) + '_step_lookahead_Hybrid'))

        
        yhat_train = model.predict(X_train, verbose=1)
        yhat_val = model.predict(X_val, verbose=1)
        y_hat_list.append(yhat_val)

        train_loss_list.append(train_history.history['loss'])
        val_loss_list.append(train_history.history['val_loss'])
        pd.DataFrame(train_loss_list).to_csv('Hybrid Univarite2/train_loss')
        pd.DataFrame(val_loss_list).to_csv('Hybrid Univarite2/val_loss')
        
        training_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_train, y_train))
        valdiation_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_val, y_val))

        # add output to next lookaheads training/val data
        x_train_extended = []
        for j in range(0, len(X_train)):
            X_train_t = np.append(X_train[j], yhat_train[j])
            x_train_extended.append(X_train_t)

        x_val_extended = []
        for k in range(0, len(X_val)):
            X_val_t = np.append(X_val[k], yhat_val[k])
            x_val_extended.append(X_val_t)
            
        #generate new targets
        n_step_lookahead = i+1
        X_traindummy, y_train, X_valdummy, y_val, scaler = generate_training_val_examples_univariate_output(data)
        
        #reshape input arrays; remove first timetep to keep dimenisons same,
        x_train = np.array(x_train_extended)
        x_val = np.array(x_val_extended)
        x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
        x_val = x_val.reshape(x_val.shape[0], x_val.shape[1],1)
        x_train = x_train[:,1:]
        x_val = x_val[:,1:]
        
        
        
        #as size of training/val window increases with increasing lookahead we lose examples
        #remove examples from training/validation inputs, keep dimensiosn same as targets
        # on reviewing the validation metrics, there is not reduction in metrics across lookaheads
        #the following two lines of code are not behaving as expected, these were changed in the 3rd iteration
        #we have sliced examples from the start of the series, not the end as intended

        X_train = x_train[-(y_train.shape[0]):,:,:]
        X_val = x_val[-(y_val.shape[0]):,:,:]
        
        np.save("Hybrid Univarite2/train_metric_dict_1.npy", training_metrics_dicts)  
        np.save("Hybrid Univarite2/val_metric_dict_1.npy", valdiation_metrics_dicts)  

        
        


D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
152/152 [==============================] - 12s 59ms/step - loss: 0.3656 - val_loss: 0.0422
Epoch 2/15
152/152 [==============================] - 8s 54ms/step - loss: 0.2887 - val_loss: 0.0380
Epoch 3/15
152/152 [==============================] - 8s 54ms/step - loss: 0.2557 - val_loss: 0.0358
Epoch 4/15
152/152 [==============================] - 8s 54ms/step - loss: 0.2341 - val_loss: 0.0327
Epoch 5/15
152/152 [==============================] - 8s 54ms/step - loss: 0.2175 - val_loss: 0.0318
Epoch 6/15
152/152 [==============================] - 8s 54ms/step - loss: 0.2073 - val_loss: 0.0298
Epoch 7/15
152/152 [==============================] - 9s 58ms/step - loss: 0.2105 - val_loss: 0.0317
Epoch 8/15
152/152 [==============================] - 9s 60ms/step - loss: 0.2085 - val_loss: 0.0292
Epoch 9/15
152/152 [==============================] - 9s 58ms/step - loss: 0.1922 - val_loss: 0.0295
Epoch 10/15
152/152 [==============================] - 9s 57ms/step - loss: 0.1881 - val_l

INFO:tensorflow:Assets written to: Hybrid Univarite2/month0_1_step_lookahead_Hybrid\assets


INFO:tensorflow:Assets written to: Hybrid Univarite2/month0_1_step_lookahead_Hybrid\assets


65/65 [==============================] - 1s 21ms/step


...

Epoch 1/15
151/151 [==============================] - 10s 55ms/step - loss: 1.6064 - val_loss: 0.0935
Epoch 2/15
151/151 [==============================] - 8s 52ms/step - loss: 1.5543 - val_loss: 0.0831
Epoch 3/15
151/151 [==============================] - 8s 52ms/step - loss: 1.5415 - val_loss: 0.0707
Epoch 4/15
151/151 [==============================] - 8s 52ms/step - loss: 1.5333 - val_loss: 0.0646
Epoch 5/15
151/151 [==============================] - 8s 52ms/step - loss: 1.5267 - val_loss: 0.0589
Epoch 6/15
151/151 [==============================] - 8s 51ms/step - loss: 1.5206 - val_loss: 0.0606
Epoch 7/15
151/151 [==============================] - 8s 52ms/step - loss: 1.5136 - val_loss: 0.0573
Epoch 8/15
151/151 [==============================] - 8s 52ms/step - loss: 1.5093 - val_loss: 0.0601
Epoch 9/15
151/151 [==============================] - 8s 51ms/step - loss: 1.5121 - val_loss: 0.0540
Epoch 10/15
151/151 [==============================] - 8s 51ms/step - loss: 1.5001 - val_l

INFO:tensorflow:Assets written to: Hybrid Univarite2/month4_10_step_lookahead_Hybrid\assets


INFO:tensorflow:Assets written to: Hybrid Univarite2/month4_10_step_lookahead_Hybrid\assets


65/65 [==============================] - 2s 20ms/step


D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [103]:
np.save("Hybrid Univarite2/train_metric_dict_1.npy", training_metrics_dicts)  
np.save("Hybrid Univarite2/val_metric_dict_1.npy", valdiation_metrics_dicts)  


In [4]:
val_metrics = np.load("Hybrid Univarite2/val_metric_dict_1.npy", allow_pickle=True)

In [6]:
indexes=[]
for j in range(1, 6):
    for i in range(1,11):
        indexes.append('month '+str(j)+' lookahead ' + str(i))

In [7]:
val_df = pd.DataFrame(val_metrics.reshape(val_metrics.shape[0], val_metrics.shape[2]), columns = ['RMSE', 'MSE', 'MAE', 'MAPE', 'R2'])

In [8]:
val_df.index=indexes[:len(val_df)]

In [9]:
val_df

,RMSE,MSE,MAE,MAPE,R2
month 1 lookahead 1,12.376245,153.171430,7.559685,0.108956,0.732118
month 1 lookahead 2,12.551234,157.533487,7.567140,0.108327,0.724490
month 1 lookahead 3,12.692354,161.095839,7.857314,0.112133,0.718379
month 1 lookahead 4,12.536113,157.154126,7.835077,0.114042,0.725270
month 1 lookahead 5,12.681144,160.811410,8.049060,0.118607,0.718877
month 1 lookahead 6,12.590872,158.530053,7.959245,0.117496,0.722865
month 1 lookahead 7,12.785877,163.478641,8.193432,0.121809,0.714347
month 1 lookahead 8,12.778948,163.301507,8.086656,0.117917,0.714657
month 1 lookahead 9,12.692485,161.099179,7.935059,0.114795,0.718505
month 1 lookahead 10,12.919996,166.926287,8.199559,0.118975,0.708444
